In [1]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('C:\\Users\\aksha\\OneDrive\\Desktop\\IMDB Dataset.csv')

In [6]:
df = df.iloc[:10000]

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:

df.drop_duplicates(inplace=True)

In [9]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [10]:
df['review'] = df['review'].apply(remove_tags)

In [11]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [12]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [13]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [14]:
import gensim

In [15]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [18]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))
    

In [19]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [20]:
model.build_vocab(story)

In [21]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5876278, 6212140)

In [22]:
len(model.wv.index_to_key)

31845

In [23]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [24]:
document_vector(df['review'].values[0])

array([-0.18468696,  0.5612938 ,  0.20352945,  0.2205931 , -0.10994098,
       -0.600725  ,  0.19403532,  0.9756532 , -0.35964808, -0.29054663,
       -0.26538154, -0.4241842 ,  0.11952145,  0.10846638,  0.23645425,
       -0.12384459, -0.02559106, -0.3527414 , -0.08620021, -0.63072056,
        0.03389994,  0.23863922,  0.07583445, -0.28354362, -0.34183484,
        0.03798299, -0.3239578 ,  0.04816606, -0.27119952,  0.06638232,
        0.34033728,  0.01775524,  0.18396495, -0.28439558, -0.16945305,
        0.45625892,  0.10577042, -0.31385797, -0.2615097 , -0.8025979 ,
        0.12934083, -0.2364172 ,  0.06272577, -0.13602266,  0.50812906,
       -0.18444788, -0.24336563, -0.05397017,  0.06030784,  0.29974973,
        0.04646403, -0.38035268, -0.45276797, -0.09021332, -0.11195419,
        0.24338412,  0.21232614,  0.05943599, -0.32728028,  0.08590481,
        0.0402751 ,  0.11032062,  0.01683515, -0.01730701, -0.44400316,
        0.26752758,  0.03784384,  0.18107857, -0.35195798,  0.29

In [25]:
from tqdm import tqdm

In [26]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [21:01<00:00,  7.92it/s] 


In [27]:
X = np.array(X)

In [28]:
X[0]

array([-0.18468696,  0.5612938 ,  0.20352945,  0.2205931 , -0.10994098,
       -0.600725  ,  0.19403532,  0.9756532 , -0.35964808, -0.29054663,
       -0.26538154, -0.4241842 ,  0.11952145,  0.10846638,  0.23645425,
       -0.12384459, -0.02559106, -0.3527414 , -0.08620021, -0.63072056,
        0.03389994,  0.23863922,  0.07583445, -0.28354362, -0.34183484,
        0.03798299, -0.3239578 ,  0.04816606, -0.27119952,  0.06638232,
        0.34033728,  0.01775524,  0.18396495, -0.28439558, -0.16945305,
        0.45625892,  0.10577042, -0.31385797, -0.2615097 , -0.8025979 ,
        0.12934083, -0.2364172 ,  0.06272577, -0.13602266,  0.50812906,
       -0.18444788, -0.24336563, -0.05397017,  0.06030784,  0.29974973,
        0.04646403, -0.38035268, -0.45276797, -0.09021332, -0.11195419,
        0.24338412,  0.21232614,  0.05943599, -0.32728028,  0.08590481,
        0.0402751 ,  0.11032062,  0.01683515, -0.01730701, -0.44400316,
        0.26752758,  0.03784384,  0.18107857, -0.35195798,  0.29

In [29]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [30]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [33]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7666499749624437